In [160]:
# To zhuzhu

import re
import requests
import pandas as pd
import numpy as np
from time import sleep
import bs4
from bs4 import BeautifulSoup
import pickle # for loading a d!ictionary from disk
from typing import Optional # typehint that value can also be None
from dataprep.clean import validate_country
from dataprep.clean import clean_country

In [161]:
#!pip install dataprep

In [162]:
import spacy
from spacy import displacy
import warnings
warnings.filterwarnings("ignore")

# Larger and slower pipeline but more accurate
# spacy.cli.download("en_core_web_trf")
# NER= spacy.load("en_core_web_trf")
                   
# Faster and smaller pipelien, less accurate                   
spacy.cli.download("en_core_web_sm")
NER= spacy.load("en_core_web_sm")

def extract_country_from_abstract(abs_string):
    text_ner = NER(abs_string)
    #displacy.render(text_ner,style="ent",jupyter=True)

    # Extract the text from each tag
    tag_list = np.array([X.text for X in text_ner.ents])
 #   tag_list_df = pd.DataFrame({ "country": tag_list })
  #  tag_list_clean = clean_country(tag_list_df, "country").country_clean.values
    GPE_mask = np.array([X.label_ =='GPE'  for X in text_ner.ents])
    LOC_mask= np.array([X.label_ =='LOC'  for X in text_ner.ents])
    mask = [validate_country(tag) for tag in tag_list]

    # Abstract_valid_country
    abs_country = tag_list[mask]
    #abs_GPE 
    GPE_list,LOC_list = [],[]
    if len(GPE_mask)>0:
      GPE_list = tag_list[GPE_mask]
    if len(LOC_mask)>0:
      LOC_list = tag_list[LOC_mask]
    return abs_country, GPE_list, LOC_list

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [163]:
def read_journal(url_list):
    cookies = dict(name='jerry', password='888')
    res = pd.DataFrame(columns= ['DOI','Journal','Type','Title','Publish_Time','Issue_Time','Co_Author','Author','Rank','Nationality','Abs_country','Abs_GPE','Abs_LOC'])
    i=0
    for url in url_list:
        home_page = requests.get(url,cookies=cookies)
        raw_html = home_page.text
        soup = BeautifulSoup(raw_html, 'html.parser')
        
        journal = soup.find('div',{'class':"row crumbs-row"}).find_all('li',{'class':"page-breadcrumbs__item"})[2].find_all('span')[-1].get_text()
        # create a new row for each author
        main_content = soup.find('div',{'id':"maincontent"})
        title = main_content.find('h1').get_text()
        contributors_details = main_content.find_all('div',{'class':'row author'})
        published_date = main_content.find('div',{'class':"row published-date"}).find('strong').get_text()

        # Extract country information from abstract
        abs_sec = soup.find('div',{'id':"app-tabs"}).find('div',{'id':'article-tab'}).find('div',{'id':'sec0','class':'col article-abstract sec'})
        
        if type(abs_sec)== bs4.element.Tag:
          abs_text = ''
          if type(abs_sec.find('div',{'class':'abstract-graphical'})) == bs4.element.Tag:
            app_tab = abs_sec.find('div',{'class':'abstract-graphical__text'})
            abs_tag = app_tab.find('div',{'class':'abstract','data-abstract-type':'normal'})
            abs_text = abs_tag.find('p').get_text()
          elif type(abs_sec.find('div',{'class':'abstract-text-container'})) == bs4.element.Tag:
            abs_text = abs_sec.find('div',{'lang':'en'}).find('div',{'class':'abstract'}).find('p').get_text()
          print(abs_text)
          abs_country, GPE_list, LOC_list = extract_country_from_abstract(abs_text)


        else:
          abs_country, GPE_list, LOC_list = [],[],[]
        
        article_details = soup.find('dl',{'class':"article-details"})
        Type = article_details.find('div',{'class':'row'}).find('dd').get_text()
        issue_time = article_details.find('div',{'class':'content__journal'}).find_all('span')[-2].get_text().strip(',')

        doi = article_details.find('div',{'class':"doi-data"}).find('span').get_text()

        for author_describe_idx in range(len(contributors_details)):
            author_describe = contributors_details[author_describe_idx]
            temp_dict={}
            temp_dict['Co_Author'] = len(contributors_details) > 1
            temp_dict['Author']=author_describe.get('data-test-author')
            temp_dict['Rank'] = 'First' if (author_describe_idx == 0) else 'Communication' \
                                if (author_describe_idx == len(contributors_details)-1) else 'Other'
            temp_dict['Nationality'] = author_describe.find_all('span')[-1].get_text().split(',')[-1].strip()
            temp_dict['Publish_Time']= published_date
            temp_dict['Issue_Time']= issue_time
            temp_dict['Title']= title
            temp_dict['Journal']= journal
            temp_dict['DOI']= doi
            temp_dict['Type']= Type
            temp_dict['Abs_country'] = abs_country
            temp_dict['Abs_GPE'] = GPE_list
            temp_dict['Abs_LOC'] = LOC_list
            res = res.append(temp_dict, ignore_index=True)
        i+=1
        print(f'Processing the {i}th paper with {len(contributors_details)} authors')
    return res

In [164]:
url = 'https://www.cambridge.org/core/journals/antiquity/all-issues'
base_url = 'https://www.cambridge.org'

In [165]:
def Antiquity_issues_url(url,base_url): 
    issues_list= [] 
    home_page = requests.get(url)
    raw_html = home_page.text
    soup = BeautifulSoup(raw_html, 'html.parser')
    all_issues = soup.find('div',{'class':'reading-width journal-all-issues'})
    Time_range = ['decade2020','decade2010','decade2000']
    for dec in Time_range:
        panels = all_issues.find('div',{'id':dec}).find('ul',{'class':'accordion level second'}).find_all('li',recursive=False)
        for volume in panels:
            issues = volume.find('div',recursive=False).find_all('a')
            for issue in issues:
                issue_url = base_url + issue.get('href')
                issues_list.append(issue_url)
    return issues_list


In [166]:
issues_url = Antiquity_issues_url(url,base_url)

In [167]:
def Antiquity_papers_url(issues_url):
    url_list=[]
    i=0
    for url in issues_url:
        home_page = requests.get(url)
        raw_html = home_page.text
        soup = BeautifulSoup(raw_html, 'html.parser')
        if soup.find('h4',text ='Project Gallery' ) != None:
            target = soup.find('h4',text ='Project Gallery' )
            for e in target.find_all_next():
                e.clear()
        elif soup.find('h4',text ='Review Article' ) != None:
            target = soup.find('h4',text ='Review Article' )
            for e in target.find_all_next():
                e.clear()
        elif soup.find('h4',text ='Book Reviews' ) != None:
            target = soup.find('h4',text ='Book Reviews' )
            for e in target.find_all_next():
                e.clear()
                
        elif soup.find('h4',text ='Method' ) != None:
            target = soup.find('h4',text ='Method' )
            for e in target.find_all_next():
                e.clear()
        elif soup.find('h4',text ='Debate' ) != None:
            target = soup.find('h4',text ='Debate' )
            for e in target.find_all_next():
                e.clear()
                
        elif soup.find('h4',text ='Project Gallery' ) != None:
            target = soup.find('h4',text ='Project Gallery' )
            for e in target.find_all_next():
                e.clear()
                
        elif soup.find('h4',text ='Front Cover (OFC, IFC) and matter' ) != None:
            target = soup.find('h4',text ='Front Cover (OFC, IFC) and matter' )
            for e in target.find_all_next():
                e.clear()
                
        elif soup.find('h4',text ='Back Cover (IBC, OBC) and matter' ) != None:
            target = soup.find('h4',text ='Back Cover (IBC, OBC) and matter' )
            for e in target.find_all_next():
                e.clear()      
                
        res = soup.find('div',{'class':'results'}).find_all('div',{'class':'representation overview search'})
        for paper in res:
            try:
                add_url = paper.find('li',{'class':'title'}).find('a').get('href')
                url_list.append(base_url+add_url)
                i+=1
                print(f'Appeding the {i}th url')
            except:
                pass
    return url_list
        

In [169]:
url_list = Antiquity_papers_url(issues_url)

Appeding the 1th url
Appeding the 2th url
Appeding the 3th url
Appeding the 4th url
Appeding the 5th url
Appeding the 6th url
Appeding the 7th url
Appeding the 8th url
Appeding the 9th url
Appeding the 10th url
Appeding the 11th url
Appeding the 12th url
Appeding the 13th url
Appeding the 14th url
Appeding the 15th url
Appeding the 16th url
Appeding the 17th url
Appeding the 18th url
Appeding the 19th url
Appeding the 20th url
Appeding the 21th url
Appeding the 22th url
Appeding the 23th url
Appeding the 24th url
Appeding the 25th url
Appeding the 26th url
Appeding the 27th url
Appeding the 28th url
Appeding the 29th url
Appeding the 30th url
Appeding the 31th url
Appeding the 32th url
Appeding the 33th url
Appeding the 34th url
Appeding the 35th url
Appeding the 36th url
Appeding the 37th url
Appeding the 38th url
Appeding the 39th url
Appeding the 40th url
Appeding the 41th url
Appeding the 42th url
Appeding the 43th url
Appeding the 44th url
Appeding the 45th url
Appeding the 46th u

In [170]:
df=read_journal(url_list)


Processing the 1th paper with 1 authors
Established chronologies indicate a long-term ‘Hoabinhian’ hunter-gatherer occupation of Mainland Southeast Asia during the Terminal Pleistocene to Mid-Holocene (45 000–3000 years ago). Here, the authors re-examine the ‘Hoabinhian’ sequence from north-west Thailand using new radiocarbon and luminescence data from Spirit Cave, Steep Cliff Cave and Banyan Valley Cave. The results indicate that hunter-gatherers exploited this ecologically diverse region throughout the Terminal Pleistocene and the Pleistocene–Holocene transition, and into the period during which agricultural lifeways emerged in the Holocene. Hunter-gatherers did not abandon this highland region of Thailand during periods of environmental and socioeconomic change.
Processing the 2th paper with 11 authors
Common assumptions about the ephemeral archaeological signature of pastoralist settlements have limited the application of geophysical techniques in the investigation of past herding

In [171]:
df.to_csv('Antiquity.csv')  

In [172]:
df

,DOI,Journal,Type,Title,Publish_Time,Issue_Time,Co_Author,Author,Rank,Nationality,Abs_country,Abs_GPE,Abs_LOC
0,https://doi.org/10.15184/aqy.2022.29,Antiquity,Editorial,Editorial,11 April 2022,April 2022,False,Robert Witcher,First,1 April 2022,[],[],[]
1,https://doi.org/10.15184/aqy.2021.44,Antiquity,Research Article,Re-evaluating Pleistocene–Holocene occupation ...,08 October 2021,April 2022,True,Cyler Conrad,First,USA,"[Thailand, Thailand]","[Thailand, Thailand]",[Mainland Southeast Asia]
2,https://doi.org/10.15184/aqy.2021.44,Antiquity,Research Article,Re-evaluating Pleistocene–Holocene occupation ...,08 October 2021,April 2022,True,Rasmi Shoocongdej,Other,Thailand,"[Thailand, Thailand]","[Thailand, Thailand]",[Mainland Southeast Asia]
3,https://doi.org/10.15184/aqy.2021.44,Antiquity,Research Article,Re-evaluating Pleistocene–Holocene occupation ...,08 October 2021,April 2022,True,Ben Marwick,Other,USA,"[Thailand, Thailand]","[Thailand, Thailand]",[Mainland Southeast Asia]
4,https://doi.org/10.15184/aqy.2021.44,Antiquity,Research Article,Re-evaluating Pleistocene–Holocene occupation ...,08 October 2021,April 2022,True,Joyce C. White,Other,USA,"[Thailand, Thailand]","[Thailand, Thailand]",[Mainland Southeast Asia]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5136,https://doi.org/10.1017/S0003598X00066138,Antiquity,News and Notes,The use of henbane (Hyoscyamus niger L.) as a ...,02 January 2015,March 2000,True,M. J. Bunting,Other,England.,[],[],[]
5137,https://doi.org/10.1017/S0003598X00066138,Antiquity,News and Notes,The use of henbane (Hyoscyamus niger L.) as a ...,02 January 2015,March 2000,True,P. Milburn,Communication,Scotland,[],[],[]
5138,https://doi.org/10.1017/S0003598X0006614X,Antiquity,News and Notes,"Taiwan, Neolithic seafaring and Austronesian o...",02 January 2015,March 2000,True,Barry V. Rolett,First,USA,[],[],"[the Taiwan Strait, Pacific]"
5139,https://doi.org/10.1017/S0003598X0006614X,Antiquity,News and Notes,"Taiwan, Neolithic seafaring and Austronesian o...",02 January 2015,March 2000,True,Wei-Chun Chen,Other,Taiwan R.O.C.,[],[],"[the Taiwan Strait, Pacific]"
